In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go



Getting NASDAQ weekly data

In [2]:
symbol = "NQ=F"

nq_weekly_df = yf.download(symbol, period="max", interval="1wk")
nq_weekly_df.reset_index(inplace=True)
nq_weekly_df.index = pd.to_datetime(nq_weekly_df["Date"], errors="coerce")

nq_weekly_df

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2000-09-18,2000-09-18,3721.00,3874.00,3602.00,3776.00,3776.00,348500
2000-09-25,2000-09-25,3776.50,3830.00,3580.50,3621.00,3621.00,310630
2000-10-02,2000-10-02,3621.00,3668.00,3305.00,3357.50,3357.50,358503
2000-10-09,2000-10-09,3359.50,3402.00,3021.50,3270.50,3270.50,368860
2000-10-16,2000-10-16,3274.00,3555.50,3025.00,3499.50,3499.50,349139
...,...,...,...,...,...,...,...
2023-09-25,2023-09-25,14880.25,15060.75,14586.00,14866.50,14866.50,3432689
2023-10-02,2023-10-02,14965.00,15160.25,14589.00,15112.00,15112.00,3846180
2023-10-09,2023-10-09,15040.25,15468.75,14949.50,15120.25,15120.25,3405149


Getting the Bias of the week. Bias = Bullish or Bearish candle

In [3]:
def get_weekly_bias(x):
    if x["Close"] < x["Open"]:
        return -1
    elif x["Close"] > x["Open"]:
        return 1

In [4]:
nq_weekly_df['weekly_bias'] = nq_weekly_df.apply(get_weekly_bias, axis=1)

nq_weekly_df.fillna(1, inplace=True)

nq_weekly_df

,Date,Open,High,Low,Close,Adj Close,Volume,weekly_bias
Date,,,,,,,,
2000-09-18,2000-09-18,3721.00,3874.00,3602.00,3776.00,3776.00,348500,1.0
2000-09-25,2000-09-25,3776.50,3830.00,3580.50,3621.00,3621.00,310630,-1.0
2000-10-02,2000-10-02,3621.00,3668.00,3305.00,3357.50,3357.50,358503,-1.0
2000-10-09,2000-10-09,3359.50,3402.00,3021.50,3270.50,3270.50,368860,-1.0
2000-10-16,2000-10-16,3274.00,3555.50,3025.00,3499.50,3499.50,349139,1.0
...,...,...,...,...,...,...,...,...
2023-09-25,2023-09-25,14880.25,15060.75,14586.00,14866.50,14866.50,3432689,-1.0
2023-10-02,2023-10-02,14965.00,15160.25,14589.00,15112.00,15112.00,3846180,1.0
2023-10-09,2023-10-09,15040.25,15468.75,14949.50,15120.25,15120.25,3405149,1.0


In [5]:
def calculate_high_low_of_week_day(nq):
    days_offset = 5
    week_start_date_object = datetime.strptime(str(nq["Date"]), "%Y-%m-%d %H:%M:%S")
    week_end_date_object = week_start_date_object + timedelta(days=days_offset)
    nq_daily_df = yf.download(
        symbol,
        start=week_start_date_object,
        end=week_end_date_object,
        interval="1d",
        progress=False,
    )
    nq_daily_df.fillna(1, inplace=True)

    nq_daily_df.reset_index(inplace=True)

    if nq["weekly_bias"] == 1:
        day_of_the_week_list = [
            nq_daily_df[nq_daily_df["Low"] == nq["Low"]]["Date"]
            .dt.strftime("%A")
            .values
        ][0].tolist()
        if len(day_of_the_week_list):
            return day_of_the_week_list[0]
    elif nq["weekly_bias"] == -1:
        day_of_the_week_list = [
            nq_daily_df[nq_daily_df["High"] == nq["High"]]["Date"]
            .dt.strftime("%A")
            .values
        ][0].tolist()
        if len(day_of_the_week_list):
            return day_of_the_week_list[0]

In [6]:
nq_weekly_df["week_high_low_day"] = nq_weekly_df.apply(
    calculate_high_low_of_week_day, axis=1
)
nq_weekly_df.fillna("Monday", inplace=True)

nq_weekly_df


,Date,Open,High,Low,Close,Adj Close,Volume,weekly_bias,week_high_low_day
Date,,,,,,,,,
2000-09-18,2000-09-18,3721.00,3874.00,3602.00,3776.00,3776.00,348500,1.0,Monday
2000-09-25,2000-09-25,3776.50,3830.00,3580.50,3621.00,3621.00,310630,-1.0,Monday
2000-10-02,2000-10-02,3621.00,3668.00,3305.00,3357.50,3357.50,358503,-1.0,Monday
2000-10-09,2000-10-09,3359.50,3402.00,3021.50,3270.50,3270.50,368860,-1.0,Monday
2000-10-16,2000-10-16,3274.00,3555.50,3025.00,3499.50,3499.50,349139,1.0,Wednesday
...,...,...,...,...,...,...,...,...,...
2023-09-25,2023-09-25,14880.25,15060.75,14586.00,14866.50,14866.50,3432689,-1.0,Friday
2023-10-02,2023-10-02,14965.00,15160.25,14589.00,15112.00,15112.00,3846180,1.0,Wednesday
2023-10-09,2023-10-09,15040.25,15468.75,14949.50,15120.25,15120.25,3405149,1.0,Monday


In [7]:
nq_weekly_df.to_csv(f'./csv/weekly_{symbol}.csv')


In [8]:
bullish_days = nq_weekly_df.loc[nq_weekly_df['weekly_bias'] == 1]
bearish_days = nq_weekly_df.loc[nq_weekly_df['weekly_bias'] == -1]


fig = go.Figure()
fig.add_trace(go.Histogram(histfunc="count", y=bullish_days["weekly_bias"], x=bullish_days["week_high_low_day"], name="weekly_bullish_count"))
fig.add_trace(go.Histogram(histfunc="count", y=bearish_days["weekly_bias"], x=bearish_days["week_high_low_day"], name="weekly_bearish_count"))

fig.show()

In [9]:


nq_daily_df = yf.download(symbol, period="max", interval="1d")
nq_daily_df.reset_index(inplace=True)
nq_daily_df.index = pd.to_datetime(nq_daily_df["Date"], errors="coerce")



[*********************100%%**********************]  1 of 1 completed


In [10]:

nq_daily_df['dr'] = nq_daily_df.apply(lambda x: x["High"] - x["Low"], axis=1)

nq_daily_df['adr'] = nq_daily_df['dr'].rolling(20).mean()

nq_daily_df



,Date,Open,High,Low,Close,Adj Close,Volume,dr,adr
Date,,,,,,,,,
2000-09-18,2000-09-18,3721.00,3785.50,3602.00,3653.50,3653.50,72994,183.50,NaN
2000-09-19,2000-09-19,3658.50,3817.00,3656.00,3804.00,3804.00,70029,161.00,NaN
2000-09-20,2000-09-20,3801.50,3874.00,3715.50,3853.00,3853.00,78832,158.50,NaN
2000-09-21,2000-09-21,3852.00,3874.00,3744.00,3765.00,3765.00,70719,130.00,NaN
2000-09-22,2000-09-22,3673.00,3780.00,3615.00,3776.00,3776.00,55926,165.00,NaN
...,...,...,...,...,...,...,...,...,...
2023-10-23,2023-10-23,14680.00,14853.75,14514.25,14712.00,14712.00,765733,339.50,280.7500
2023-10-24,2023-10-24,14744.00,14948.75,14706.00,14846.50,14846.50,703058,242.75,278.7500
2023-10-25,2023-10-25,14828.00,14855.00,14444.00,14478.25,14478.25,820677,411.00,287.5250


In [11]:
px.scatter(data_frame=nq_daily_df, x="Date", y='adr', trendline='ols' )

c:\Users\praveenjaisankar\AppData\Local\pypoetry\Cache\virtualenvs\metatrader-Cdlomroj-py3.10\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\praveenjaisankar\AppData\Local\pypoetry\Cache\virtualenvs\metatrader-Cdlomroj-py3.10\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

